In [1]:
#modifcado: Víctor V.
#importing the dependencies
#genera texto a partir de Alicia en el país de las marvillas.
import numpy
#redes neuronales secuenciales
from keras.models import Sequential
#capa fuertemente conectada, densa
from keras.layers import Dense
#tecnica para recortar y hacer que aprenda de nuevo
#sirve para evitar sobre entrenamiento
from keras.layers import Dropout
#rede neuronal LSTM(Long Short-Term Memory)
from keras.layers import LSTM
#Esta clase es esencial para guardar el estado de tu modelo durante el entrenamiento.
from keras.callbacks import ModelCheckpoint
#from keras.utils import np_utils
#Codificación One-Hot: Es una técnica para representar variables categóricas como vectores numéricos.
from sklearn.preprocessing import OneHotEncoder
#one-Hot
from tensorflow.keras.utils import to_categorical
import sys

In [37]:
# sube y lo convierte a minúscua
#filename = "wonderland.txt"
filename = "dinos.csv"

#leer el texto
raw_text = open(filename).read()
#pasar el texto a minúscula
raw_text = raw_text.lower()

In [38]:
#crea un diccionario con caracteres únicos del lenguaje
#con su representación en número, diccionaro
chars = sorted(list(set(raw_text)))


In [39]:
# here we can see all the unique charecters in the book
print(chars)

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [40]:
#carcter y su índice
#colca en un diccionario la letra y una posición
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [41]:
#realiza conteos
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  9668
Total Vocab:  27


In [42]:
# prepara el data set para el part de entrada salida
seq_length = 100
dataX = []
dataY = []
#ciclo que aumenta de 1 en 1
for i in range(0, n_chars - seq_length, 1):
    #una porción de código 100
    seq_in = raw_text[i:i + seq_length]
    #print('seq in : ', seq_in)
    #caracter a caracter
    seq_out = raw_text[i + seq_length]
    #print('seq out : ', seq_out)

    dataX.append([char_to_int[char] for char in seq_in])
    #print(dataX)
    #número
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  9568


In [43]:
print(dataY)

[3, 18, 9, 19, 20, 1, 22, 21, 19, 0, 1, 3, 18, 15, 3, 1, 14, 20, 8, 15, 19, 1, 21, 18, 21, 19, 0, 1, 3, 18, 15, 20, 8, 15, 12, 21, 19, 0, 1, 4, 1, 13, 1, 14, 20, 9, 19, 1, 21, 18, 21, 19, 0, 1, 4, 1, 19, 1, 21, 18, 21, 19, 0, 1, 4, 5, 15, 16, 1, 16, 16, 15, 19, 1, 21, 18, 21, 19, 0, 1, 5, 7, 25, 16, 20, 15, 19, 1, 21, 18, 21, 19, 0, 1, 5, 15, 12, 15, 19, 1, 21, 18, 21, 19, 0, 1, 5, 18, 15, 19, 20, 5, 15, 14, 0, 1, 6, 18, 15, 22, 5, 14, 1, 20, 15, 18, 0, 1, 7, 1, 20, 8, 1, 21, 13, 1, 19, 0, 1, 7, 9, 12, 9, 19, 1, 21, 18, 21, 19, 0, 1, 7, 21, 10, 1, 3, 5, 18, 1, 20, 15, 16, 19, 0, 1, 7, 21, 19, 20, 9, 14, 9, 1, 0, 1, 10, 11, 1, 3, 5, 18, 1, 20, 15, 16, 19, 0, 1, 12, 1, 13, 15, 19, 1, 21, 18, 21, 19, 0, 1, 12, 1, 19, 11, 1, 3, 5, 16, 8, 1, 12, 5, 0, 1, 12, 2, 1, 12, 15, 16, 8, 15, 19, 1, 21, 18, 21, 19, 0, 1, 12, 2, 5, 18, 20, 1, 3, 5, 18, 1, 20, 15, 16, 19, 0, 1, 12, 2, 5, 18, 20, 1, 4, 18, 15, 13, 5, 21, 19, 0, 1, 12, 2, 5, 18, 20, 15, 14, 25, 11, 21, 19, 0, 1, 12, 2, 5, 18, 20, 15, 19,

In [44]:
# cambia X a  [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# representación one-hot
y = to_categorical(dataY)
X,y

(array([[[0.03703704],
         [0.03703704],
         [0.66666667],
         ...,
         [0.66666667],
         [0.        ],
         [0.03703704]],
 
        [[0.03703704],
         [0.66666667],
         [0.14814815],
         ...,
         [0.        ],
         [0.03703704],
         [0.11111111]],
 
        [[0.66666667],
         [0.14814815],
         [0.55555556],
         ...,
         [0.03703704],
         [0.11111111],
         [0.66666667]],
 
        ...,
 
        [[0.55555556],
         [0.51851852],
         [0.25925926],
         ...,
         [0.03703704],
         [0.77777778],
         [0.66666667]],
 
        [[0.51851852],
         [0.25925926],
         [0.        ],
         ...,
         [0.77777778],
         [0.66666667],
         [0.77777778]],
 
        [[0.25925926],
         [0.        ],
         [0.96296296],
         ...,
         [0.66666667],
         [0.77777778],
         [0.7037037 ]]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0.,

In [45]:

# define el LSTM model
#Red secuencia
model = Sequential()
#agrega una capa al modelo secuencial
#Define una capa LSTM con 256 unidades ocultas.
#input_shape=(X.shape[1], X.shape[2]): Especifica la forma de entrada esperada para la capa LSTM.
#X.shape[1]Longitud de las secuencias (número de pasos de tiempo).
#X.shape[2] Número de características en cada paso de tiempo.
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#corte brusco,Dropout es una técnica de regularización utilizada para prevenir el sobreajuste en redes neuronales
model.add(Dropout(0.2))
#capa Densa con la salida igual al tamaño de y ,activacion softmax
model.add(Dense(y.shape[1], activation='softmax'))
#cómo va a configurar la entropía,optimizador Adam redes neuronales, medida
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [46]:
#descripción de lared
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 256)               264192    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 27)                6939      
                                                                 
Total params: 271131 (1.03 MB)
Trainable params: 271131 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
# define the punto de verificación
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras" # Add .keras extension
#save_best_only=True: Solo guarda el modelo con el mejor valor de la métrica
#mode='min': Indica que se busca minimizar el valor de la métrica monitoreada (la pérdida).
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]



In [48]:
# entrena el modelo
#callbacks=callbacks_list: Lista de objetos de devolución de llamada (callbacks) para monitorear y modificar el entrenamiento.
model.fit(X, y, epochs=3, batch_size=2000, callbacks=callbacks_list)
#epochs=10, cuántas veces le a pasar el conjunto de entrenamiento
#model.fit(X, y, epochs=10, batch_size=2000, callbacks=callbacks_list)

Epoch 1/3
5/5 [==============================] - ETA: 0s - loss: 3.2216 - accuracy: 0.1109
Epoch 1: loss improved from inf to 3.22165, saving model to weights-improvement-01-3.2216.keras
5/5 [==============================] - 7s 1s/step - loss: 3.2216 - accuracy: 0.1109
Epoch 2/3
5/5 [==============================] - ETA: 0s - loss: 2.9185 - accuracy: 0.1159
Epoch 2: loss improved from 3.22165 to 2.91845, saving model to weights-improvement-02-2.9185.keras
5/5 [==============================] - 5s 1s/step - loss: 2.9185 - accuracy: 0.1159
Epoch 3/3
5/5 [==============================] - ETA: 0s - loss: 2.8586 - accuracy: 0.1050
Epoch 3: loss improved from 2.91845 to 2.85863, saving model to weights-improvement-03-2.8586.keras
5/5 [==============================] - 5s 993ms/step - loss: 2.8586 - accuracy: 0.1050


In [49]:
#guarda el modelo
model.save('model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
#guarda la red con pesos
#from keras.models import load_model
#"Esta línea de código en Python carga un modelo previamente guardado en un archivo HDF5.
#model =""
#model = load_model('saved weights small network/weights-improvement-20-2.6334.hdf5')

OSError: No file or directory found at saved weights small network/weights-improvement-20-2.6334.hdf5

In [50]:
#de caractres con índices, crea un diccionario(índice,caracter)
int_to_char = dict((i, c) for i, c in enumerate(chars))
#vector que me permite  recuperar la letra por el índice, posición
int_to_char

{0: '\n',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

In [51]:
dataX

[[1,
  1,
  18,
  4,
  15,
  14,
  25,
  24,
  0,
  1,
  2,
  5,
  12,
  9,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  2,
  18,
  9,
  3,
  20,
  15,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  2,
  18,
  15,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  2,
  25,
  4,
  15,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  3,
  1,
  14,
  20,
  8,
  15,
  16,
  8,
  15,
  12,
  9,
  19,
  0,
  1,
  3,
  8,
  5,
  12,
  15,
  21,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  3,
  8,
  9,
  12,
  12,
  15,
  2,
  1,
  20,
  15,
  18,
  0,
  1],
 [1,
  18,
  4,
  15,
  14,
  25,
  24,
  0,
  1,
  2,
  5,
  12,
  9,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  2,
  18,
  9,
  3,
  20,
  15,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  2,
  18,
  15,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  2,
  25,
  4,
  15,
  19,
  1,
  21,
  18,
  21,
  19,
  0,
  1,
  3,
  1,
  14,
  20,
  8,
  15,
  16,
  8,
  15,
  12,
  9,
  19,
  0,
  1,
  3,
  8,
  5,
  12,
  15,
  21,
  19,
  1,
 

In [52]:
int_to_char[11]

'k'

In [53]:
#generación de texto
#aquí la red neuronal aprendió del texto de Alicia

# selecciona un índice del tamaño del datos
start = numpy.random.randint(0, len(dataX)-1)
#print(dataX)
start = 0
#inicia con el texto en indice calculado
pattern = dataX[start]
print ("Semilla:")
#obtiene el caracter de la posición
print ("\"", ' '.join([int_to_char[value] for value in pattern]), "\"")

# generar caracteres
for i in range(1000):
	# redimensiona un array pattern a una nueva forma tridimensional
	#1: El primer elemento indica que se crea un nuevo eje con una sola dimensión.
	#El segundo elemento especifica que la segunda dimensión tendrá la misma longitud que el array original pattern.
	#1: El tercer elemento indica que se crea un nuevo eje con una sola dimensión.
	x = numpy.reshape(pattern, (1, len(pattern), 1))
  #divide cada elemento del array x por el valor de n_vocab convertido a flotante.
	x = x / float(n_vocab)
 #realiza la predicción,no mostrando nada,verbose=1
	prediction = model.predict(x, verbose=0)
  #obtiene el índice del valor más alto en la predicciónms
	index = numpy.argmax(prediction)
  #el resultado lo coloca como índice y saca el caracter
	result = int_to_char[index]
  #guarda el caracter en una lista, de número a lista
	seq_in = [int_to_char[value] for value in pattern]
  #escribe el resultado
	sys.stdout.write(result)

  #adiciona el índice a patter
	pattern.append(index)
 #pattern a una nueva lista que contiene todos los elementos de la lista original pattern excepto el primer elemento.
	pattern = pattern[1:len(pattern)]
print(result)
print ("\nDone.")

Semilla:
" a a r d o n y x 
 a b e l i s a u r u s 
 a b r i c t o s a u r u s 
 a b r o s a u r u s 
 a b y d o s a u r u s 
 a c a n t h o p h o l i s 
 a c h e l o u s a u r u s 
 a c h i l l o b a t o r 
 a "
a

Done.


here we can see that the model performs very badly. an educated guess for this is that the model is too simple and is also trained for smaller epochs

In [54]:
# training larger model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [55]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 100, 256)          264192    
                                                                 
 dropout_5 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_6 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 27)                6939      
                                                                 
Total params: 796443 (3.04 MB)
Trainable params: 796443 (3.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=10, batch_size=1300, callbacks=callbacks_list)

Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 3.0682 - accuracy: 0.1045
Epoch 1: loss improved from inf to 3.06822, saving model to weights-improvement-01-3.0682-bigger.hdf5
8/8 [==============================] - 18s 2s/step - loss: 3.0682 - accuracy: 0.1045
Epoch 2/10
8/8 [==============================] - ETA: 0s - loss: 2.8675 - accuracy: 0.1142
Epoch 2: loss improved from 3.06822 to 2.86749, saving model to weights-improvement-02-2.8675-bigger.hdf5
8/8 [==============================] - 15s 2s/step - loss: 2.8675 - accuracy: 0.1142
Epoch 3/10
8/8 [==============================] - ETA: 0s - loss: 2.8366 - accuracy: 0.1137
Epoch 3: loss improved from 2.86749 to 2.83659, saving model to weights-improvement-03-2.8366-bigger.hdf5
8/8 [==============================] - 15s 2s/step - loss: 2.8366 - accuracy: 0.1137
Epoch 4/10
8/8 [==============================] - ETA: 0s - loss: 2.8326 - accuracy: 0.1237
Epoch 4: loss improved from 2.83659 to 2.83258, saving model to

In [ ]:
#training the model for another round of 10 epochs
model.fit(X, y, epochs=10, batch_size=1300, callbacks=callbacks_list)

Epoch 1/10
144415/144415 [==============================] - 3275s 23ms/step - loss: 2.3322 - acc: 0.3312
Epoch 2/10
144415/144415 [==============================] - 3333s 23ms/step - loss: 2.2832 - acc: 0.3450
Epoch 3/10
144415/144415 [==============================] - 3205s 22ms/step - loss: 2.2351 - acc: 0.3579
Epoch 4/10
144415/144415 [==============================] - 3310s 23ms/step - loss: 2.1945 - acc: 0.3696
Epoch 5/10
144415/144415 [==============================] - 3150s 22ms/step - loss: 2.1582 - acc: 0.3801
Epoch 6/10
144415/144415 [==============================] - 3139s 22ms/step - loss: 2.1219 - acc: 0.3895
Epoch 7/10
144415/144415 [==============================] - 3180s 22ms/step - loss: 2.0907 - acc: 0.3963
Epoch 8/10
144415/144415 [==============================] - 3342s 23ms/step - loss: 2.0630 - acc: 0.4049
Epoch 9/10
144415/144415 [==============================] - 3251s 23ms/step - loss: 2.0331 - acc: 0.4122
Epoch 10/10
144415/144415 [============================

In [36]:
#generating the text
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" it; so, after hunting all about for it, he was obliged to write with one finger for the rest of the  "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

We can see that generally there are fewer spelling mistakes and the text looks more realistic, but is still quite nonsensical.
try fitting a more complex network or try training this network for longer epochs in order to see considerable differences